# Train DeepMind GNN Physics Simulator
This notebook automatically provisions a Kaggle GPU Environment to run the DeepMind Graph Neural Network Physics Simulator optimization pipeline natively.
It pulls the complete `WaterDrop` dataset via DeepMind's bucket and trains via `train.py`.

In [ ]:
!rm -rf nans-projekat
!git clone https://github.com/MarkoMile/gnn-physics-simulator.git
%cd gnn-physics-simulator

# Install PyTorch Geometric and its dependencies aligned for Kaggle's CUDA 12 environment
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.1.2+cu121.html
!pip install torch-geometric

# Install utility dependencies (Weights & Biases, PyYAML, TF Data Parser)
!pip install wandb pyyaml tensorflow

In [ ]:
import tqdm.notebook
import tqdm.auto

# Force the auto-detector to use the notebook progressive version to avoid console flooding
tqdm.auto.tqdm = tqdm.notebook.tqdm
tqdm.auto.trange = tqdm.notebook.trange

In [ ]:
# Fetch the full multi-GB DeepMind WaterDrop dataset straight from Google Cloud
!python scripts/download_dataset.py WaterDrop

import yaml

# Patch the config.yaml dynamically to target the massive dataset instead of the tiny sample
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
config['data']['dataset_path'] = 'data/WaterDrop'
config['data']['dataset_format'] = 'tfrecord'

with open('config.yaml', 'w') as f:
    yaml.dump(config, f)
    
print("Patched config.yaml successfully to target full data/WaterDrop!")

In [ ]:
import os
import wandb
from kaggle_secrets import UserSecretsClient

try:
    user_secrets = UserSecretsClient()
    wandb_api_key = user_secrets.get_secret("wandb_api_key")
    os.environ["WANDB_API_KEY"] = wandb_api_key
    
    wandb.login()
    print("WandB logged in successfully using Kaggle Secrets.")
except Exception as e:
    print(f"WandB login skipped/failed: {e}\nEnsure you have added 'wandb_api_key' to your Kaggle Secrets if you want telemetry.")

In [ ]:
# Execute the global training loop with WandB telemetry enabled!
!python train.py --config config.yaml --wandb